# Examining FRIBDAQ Data: The Ion Chamber

This notebook demonstrates the algorithm in the point cloud phase used to analyze ion chamber data. Spyral *does not* need to have been run to use this notebook. Typically, AT-TPC data contains more than just the data produced by the AT-TPC itself. In particular, an upstream ion chamber is critical for selecting the beam of interest entering the AT-TPC. Without this, the data is polluted by reactions involving other beams than the species of interest. This data is typically handled by a separate DAQ called FRIB(NSCL)DAQ.  This notebook will demonstrate the analysis used by Spryal to extract the FRIBDAQ data as well as how it uses this data to improve the AT-TPC results.

First we load the relevant libraries

In [ ]:
from spyral.trace.frib_event import FribEvent, IC_COLUMN, SI_COLUMN
from spyral.trace.frib_trace import FRIB_TRACE_LENGTH
from spyral.phases.pointcloud_phase import get_event_range
from spyral.core.run_stacks import form_run_string
from spyral import FribParameters, DetectorParameters

import h5py as h5
import numpy.random as random
import numpy as np
from pathlib import Path
import plotly.graph_objects as go

Now we load our configuration and workspace. While using this notebook one can also customize the configuration on the fly without modifying the acutal JSON file

In [ ]:
# Set some parameters
trace_data_path = Path("/path/to/raw/attpc/traces/")

frib_params = FribParameters(
    baseline_window_scale=100.0,
    peak_separation=50.0,
    peak_prominence=20.0,
    peak_max_width=500.0,
    peak_threshold=100.0,
    ic_delay_time_bucket=1100,
    ic_multiplicity=1,
    correct_ic_time=True,
)

det_params = DetectorParameters(
    magnetic_field=2.85,
    electric_field=45000.0,
    detector_length=1000.0,
    beam_region_radius=25.0,
    micromegas_time_bucket=10.0,
    window_time_bucket=560.0,
    get_frequency=6.25,
    garfield_file_path=Path("/path/to/some/garfield.txt"),
    do_garfield_correction=False,
)


Pick a run and load the raw trace HDF5 file

In [ ]:
run_number = 16
trace_file_path = trace_data_path / f"{form_run_string(run_number)}.h5"
trace_file = h5.File(trace_file_path, "r")

We select the FRIB group and the evt subgroup (evt is an FRIBDAQ convention meaning the actual event data)

In [ ]:
frib_group: h5.Group = trace_file['frib']
trace_group: h5.Group = frib_group['evt']

Now we select a specific event from the FRIBDAQ data. The event numbers here should match the event numbers in the GET data. By default a random event is selected, but it can be useful to hardcode the event to inspect specific behavior. We then retrieve the traces from the SIS3300 module (id 1903).

In [ ]:
# Ask the trace file for the range of events
min_event, max_event = get_event_range(trace_file)
# Select a random event
event_number = random.randint(min_event, max_event)
print(f'Event {event_number}')
# Can always overwrite with hardcoded event number if needed
# event_number = 38537

trace_data: h5.Dataset = trace_group[f'evt{event_number}_1903']

First lets plot the raw trace for the ion chamber and an auxilary silicon detector

In [ ]:
sample_range = np.arange(0, FRIB_TRACE_LENGTH)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=sample_range,
        y=trace_data[:, IC_COLUMN],
        mode="lines",
        name="Ion Chamber"
    )
)
fig.add_trace(
    go.Scatter(
        x=sample_range,
        y=trace_data[:, SI_COLUMN],
        mode="lines",
        name="Silicon"
    )
)
fig.update_layout(
    xaxis_title="Time Bucket",
    yaxis_title="Amplitude"
)

Now we'll clean up those traces, removing the baseline, by passing the data to the FribEvent class. This will also identify peaks in the traces, which we'll label in the plot.

In [ ]:
event = FribEvent(trace_data, event_number, frib_params)
si_cents = []
si_amps = []
for peak in event.get_si_trace().peaks:
    si_cents.append(peak.centroid)
    si_amps.append(peak.amplitude)
ic_cents = []
ic_amps = []
for peak in event.get_ic_trace().peaks:
    ic_cents.append(peak.centroid)
    ic_amps.append(peak.amplitude)


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=sample_range,
        y=event.get_ic_trace().trace,
        mode="lines",
        name="Ion Chamber"
    )
)
fig.add_trace(
    go.Scatter(
        x=sample_range,
        y=event.get_si_trace().trace,
        mode="lines",
        name="Silicon"
    )
)
fig.add_trace(
    go.Scatter(
        x=ic_cents,
        y=ic_amps,
        mode="markers",
        name="IC Peaks"
    )
)
fig.add_trace(
    go.Scatter(
        x=si_cents,
        y=si_amps,
        mode="markers",
        name="Si Peaks"
    )
)
fig.update_layout(
    xaxis_title="Time Bucket",
    yaxis_title="Amplitude"
)

Another important concept of the ion chamber signal is the triggering peak. The ion chamber signal is used to generate the master trigger for an event, when in coincidence with the mesh signal. As the mesh signal is significantly later than the ion chamber, the ion chamber signal must be delayed. This delay means that, in the above spectra, there is no guarantee that the first peak is the peak which generated the trigger. In a basic IC analysis, the peak that we want is the peak that triggers, not all the other ones. Even in more advanced analyses we would need to know which peak caused the trigger. To figure out which peak causes the trigger, we use a simple method. Since the ion chamber is delayed a fixed amount every event, in general, the position of the trigger should not vary much. By running through some events in this notebook, you can identify this position, and use the `ic_delay_time_bucket` paramter in the configuration. This sets a lower threshold (in time); the first peak past this threshold is taken as the trigger. As such, FribEvent has some methods to handle this, demonstrated below.

In [ ]:
trigger = event.get_triggering_ic_peak(frib_params)
if trigger is None:
    raise Exception("There is no trigger in this event! Uh Oh!")

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=sample_range,
        y=event.get_ic_trace().trace,
        mode="lines",
        name="Ion Chamber"
    )
)
fig.add_trace(
    go.Scatter(
        x=[trigger.centroid],
        y=[trigger.amplitude],
        mode="markers",
        name="Triggering Peak"
    )
)
fig.add_vline(frib_params.ic_delay_time_bucket, annotation_text="Ion Chamber Delay")

Finally, we can use the peaks to identify the "good" ion chamber peak. A good ion chamber peak is identified as an ion chamber peak that *does not* have a coincident silicon peak and occurs after the trigger (or is the trigger). If the good ion chamber peak is not the trigger in the ion chamber spectrum, this means that the event time was acutally offset by the wrong beam event. We can correct for this by calculating the time difference between the trigger peak time and the good ion chamber peak time. Additionally, the configuration can controll the maximum allowed multiplicity for the ion chamber. By default the only singles events are allowed.

In [ ]:
good_ic = event.get_good_ic_peak(frib_params)
if good_ic is not None:
    peak = good_ic[1]
    mult = good_ic[0]
    print(f"Good IC Peak: {peak} Multiplicity: {mult}")
    ic_offset = event.correct_ic_time(peak, frib_params, det_params.get_frequency)
    print(f'IC Time Offset in GET Buckets: {ic_offset}')
else:
    print("No good IC peak")